In [1]:
import numpy as np
import pandas as pd
from datasketch import MinHashLSHForest, MinHash, MinHashLSH
import random
import time
import sys, io

## data preprocessing    
- remove test accounts    
- aggregate data and output csv for next step

In [96]:
test_accounts = []
with open('test_accounts.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        test_accounts.append(line.strip())
print(str(len(test_accounts))+' test accounts to be removed across all relations')

65 test accounts to be removed across all relations


In [97]:
test_accounts

['0@gmail.com',
 'adrian.ponoran@ipalapa.com',
 'adrian@ipalapa.com',
 'adrian@thefoodexchange.com',
 'adriansrep@adrian.com',
 'adrian_ponoran@tahoo.com',
 'adrian_ponoran@yahoo.com',
 'agorgoi+1@tremend.ro',
 'agorgoi@tremend.ro',
 'aidansweitz@gmail.com',
 'amanda@mailinator.com',
 'amanda@thefoodexchange.com',
 'amandatfe@mailinator.com',
 'aponoran@gmail.com',
 'google54@google.com',
 'info@thefoodexchange.com',
 'ivana.cwang@gmail.com',
 'ivana@thefoodexchange.com',
 'ivangarcia2@gmail.com',
 'jeremiah@thefoodexchange.com',
 'jon@thefoodexchange.com',
 'jondoe@gmail.com',
 'jontesting@tfe.com',
 'karthik@buildingblock.la',
 'karthik@buildingblocks.la',
 'karthik@buildingblocks.la1',
 'karthik@buildingblockshgh.com',
 'karthik@buildingblockshgh.la',
 'karthik@buildingblockshgh.lgg',
 'karthik@buildingblockssdfs.la',
 'karthik@buildingblockssdfsd.la',
 'karthik@buildingblockssdfsdf.la',
 'karthik@buildingblockswerwe.la',
 'karthik@thefoodexchnage.com',
 'karthik@thefoodexchnge.com'

In [98]:
def remove_accounts(df, by_col):
    df = df[~df[by_col].isin(test_accounts)]
    return df

In [99]:
users = pd.read_csv('tfe_user_1128.csv')
users = remove_accounts(users,'email')

In [100]:
users.shape

(631, 20)

In [20]:
users.head()

,email,account_number,account_type,company_id,cr_date,first_name,is_admin,is_email_confirmed,last_login_date,last_name,mod_date,old_company_id,password_hash,phone,secret_key,terms_accepted,token,url_id,user_key,is_disabled
1,261014mu@gmail.com,NaN,1,19203,2018-10-02 23:03:54,Caleb,0,1,2018-10-26 21:33:34,Jones,2018-10-26 21:33:34,NaN,5d42d7177b08e005292f4b146e75331b,NaN,"[-33, 42, -113, -118, 53, 88, 2, -123, -70, -2...",0,NaN,calebjones,ahVzfnRoZWZvb2RleGNoYW5nZXByb2RyHwsSB1RmZVVzZX...,0.0
2,592283977@qq.com,NaN,1,19354,2018-10-03 18:45:13,Zhaoyang,0,1,2018-10-03 18:47:07,Song,2018-10-03 18:47:07,NaN,9e1fc442081019ee1e4650833eba1a5b,NaN,"[-78, 2, 41, -37, -63, 35, 72, 79, 94, 105, -1...",0,NaN,zhaoyangsong,ahVzfnRoZWZvb2RleGNoYW5nZXByb2RyHQsSB1RmZVVzZX...,0.0
3,aaron.r@authenticfoods.com,NaN,0,5752,2017-10-09 21:47:16,Aaron,0,1,2018-10-23 23:47:16,Rice,2018-10-23 23:47:16,NaN,5ee39351c4f03c86161db9d08ff57935,NaN,"[86, 40, -7, -128, -128, -24, -3, 63, 86, -3, ...",0,NaN,aaronrice,ahVzfnRoZWZvb2RleGNoYW5nZXByb2RyJwsSB1RmZVVzZX...,0.0
4,ab.castillocamelo@gmail.com,NaN,1,4453,2017-09-15 20:51:47,Angelica,0,1,2017-09-20 22:04:30,Castillo,2017-09-20 22:04:30,NaN,9201a8f6a4480d9b609a4970a3796629,NaN,"[-38, 64, -65, 17, -36, 98, 48, -44, 26, 45, 3...",0,NaN,angelicacastillo,ahVzfnRoZWZvb2RleGNoYW5nZXByb2RyKAsSB1RmZVVzZX...,0.0
5,abayer@geniusjuice.com,NaN,0,14904,2018-06-24 18:48:11,Alex,0,1,2018-08-16 15:37:10,Bayer,2018-08-16 15:37:10,NaN,a9fa4f3bb6787b528e023f301581abe9,NaN,"[-55, -14, 109, 22, -127, 6, -77, -25, -7, 45,...",0,NaN,alexbayer,ahVzfnRoZWZvb2RleGNoYW5nZXByb2RyIwsSB1RmZVVzZX...,0.0


In [102]:
product_order = pd.read_csv('product_order_1128.csv')
product_order = remove_accounts(product_order,'cr_by')

In [103]:
product_order.shape

(577, 39)

In [104]:
product_order.head()

,id,cr_by,cr_date,mod_by,mod_date,old_id,attribute_str,available_qty,brand,category,...,uom,upc,vendor_category,vendor_commodity,vendor_id,vendor_product_code,vendor_product_number,warehouse_info,source_product_id,unit_price
0,1,jessecason@gmail.com,7/30/16 16:07,jessecason@gmail.com,7/30/16 16:07,6.508720e+15,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,54.0,NaN,NaN,NaN,NaN,NaN
1,2,fed@thecheffed.com,7/30/16 01:46,fed@thecheffed.com,7/30/16 01:46,5.544970e+15,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,106.0,NaN,NaN,NaN,NaN,NaN
2,3,nasimeh@nadertrading.com,9/22/16 18:46,nasimeh@nadertrading.com,9/22/16 18:46,5.043970e+15,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,78.0,NaN,BA5,NaN,NaN,NaN
3,4,nasimeh@nadertrading.com,9/22/16 18:46,nasimeh@nadertrading.com,9/22/16 18:46,5.221240e+15,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,78.0,NaN,OL6,NaN,NaN,NaN
4,5,jasmine@thespringsla.com,8/9/16 21:45,jasmine@thespringsla.com,8/9/16 21:45,5.594120e+15,NaN,0,Coco Bakes,NaN,...,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN,NaN,NaN


In [105]:
product_order['source_product_id'].isnull().sum()

146

In [106]:
product_order['price'] = product_order['price'].map(lambda x: float(x))
tot_transaction = product_order['price'].sum()
print('Total transaction: '+str(tot_transaction))

Total transaction: 24583.38


In [107]:
clipboard = pd.read_csv('clipboard_list_item_1128.csv')

In [108]:
clipboard = remove_accounts(clipboard,'cr_by')

In [109]:
clipboard.shape

(2742, 16)

In [115]:
clipboard.head()

,id,cr_by,cr_date,mod_by,mod_date,old_id,old_product_id,old_user_clipboard_id,old_vendor_id,on_hand,item_order,par,product_id,quantity,user_clipboard_id,vendor_id
0,1,jasmine@thespringsla.com,2016-08-09 15:00:41,NaN,NaN,4.507487e+15,5.485462e+15,5.126723e+15,5.102661e+15,0,NaN,0,1011,0,25,70
1,2,jasmine@thespringsla.com,2016-08-09 15:00:41,NaN,NaN,4.508524e+15,6.552980e+15,5.126723e+15,5.102661e+15,0,NaN,0,4271,0,25,70
2,3,chef@chefdas.com,2017-02-03 19:22:49,NaN,NaN,4.510450e+15,6.342217e+15,5.186932e+15,4.794446e+15,0,40.0,0,3576,0,39,56
3,4,mariovollera@gmail.com,2017-02-24 18:30:18,NaN,NaN,4.516822e+15,5.462643e+15,5.098710e+15,5.070161e+15,0,20.0,0,6737,0,15,60
4,5,mariovollera@gmail.com,2017-02-24 18:32:16,NaN,NaN,4.519971e+15,6.135875e+15,5.098710e+15,6.316792e+15,0,7.0,0,1841,0,15,120


In [59]:
product_dict = {}
for row in product_order.itertuples():
    try:
        product_id = int(row.source_product_id)
        product_name = row.name
        if product_id not in product_dict.keys():
            product_dict[product_id] = product_name
    except ValueError:
        pass

for row in clipboard.itertuples():
    try:
        product_id = int(row.product_id)
        product_name = row.name
        if product_id not in product_dict.keys():
            product_dict[product_id] = product_name
    except ValueError:
        pass

In [60]:
product_dict

{842: 'Bottom Sirloin - Ball Tip',
 8452: 'Cilantro Chutney',
 22242: 'Beans, Vanilla 1/2 LB',
 22258: 'Berries, Blackberries EACH',
 22211: 'Apples, Gravenstein LB',
 22241: 'Beans, Romano LB',
 22202: 'Apples, Braeburn',
 22222: 'Arugula, Baby Wild 4 LB',
 22219: 'Artichokes LB',
 23306: 'hamburger buns',
 22627: 'Conv. Herbs, Cilantro BUNCH',
 22956: 'Lettuce, Red Leaf EACH',
 22737: 'Conv. Peppers, Jalapeno LB',
 22799: 'Conv. Tomatoes, Slicer 2 layer',
 23307: 'Sweet Salad Mix 3 LB',
 22304: 'Carrots, Orange LB',
 22813: 'Cucumbers EACH',
 23309: '12 whole wheat tortillas"',
 23308: 'Tortillas, Gluten Free 12 6 CT"',
 22757: 'Conv. Potatoes, Russet 80 CTLB',
 23952: 'MEXICAN MOLOTOV CHILLI SAUCE',
 23752: 'Just Mayo Original, 4/1 Gallon',
 71: 'ANCHOVY IN OIL',
 42: 'ACHIOTE PASTE',
 43: 'ACHIOTE PASTE',
 82: 'ANISE FENNEL',
 56: 'ALLSPICE WHOLE SML BOTL',
 94: 'APPLES BRAEBURN',
 60: 'ALMOND NATURAL EXTRACT',
 24752: 'Artichokes',
 881: 'Bulgarian Sheep Milk Feta ',
 6083: 'Rock 

In [112]:
product_order_notnull = product_order[product_order['source_product_id'].notnull()]
product_order_notnull['source_product_id'] = product_order_notnull['source_product_id'].map(lambda x: int(x))
product_order_gb = product_order_notnull.groupby(['cr_by'])['source_product_id'].apply(set).to_frame().reset_index()

In [113]:
product_order_gb.shape

(39, 2)

In [171]:
product_order_gb.rename(columns={'source_product_id': 'product_id'}, inplace=True)

In [172]:
product_order_gb

,cr_by,product_id
0,ajish@techbreakthrough.co.in,"{40205, 40206}"
1,alison@pamelasproducts.com,{38198}
2,angelthehivesm@gmail.com,"{38274, 38540, 38541, 38303, 38304, 38305, 383..."
3,anh.nguyen@bistrosystems.com,{39202}
4,anna@themasoncoffee.com,"{7393, 7400, 7401, 7406, 7407}"
5,cassandra.winegard@matthewkenneycuisine.com,"{52002, 52005, 52006, 51799, 52011, 51915, 519..."
6,chris.nuciforo@outlook.com,"{35211, 35212, 35213, 35214, 35215, 35216, 352..."
7,erockey@rockeysmilkpunch.com,{52652}
8,giles@toriilabs.com,{33552}
9,griff.foxley@gmail.com,"{24704, 24705, 24676, 842, 24715, 24682, 24685..."


In [116]:
clipboard_notnull = clipboard[clipboard['product_id'].notnull()]
clipboard_gb = clipboard_notnull.groupby(['cr_by'])['product_id'].apply(set).to_frame().reset_index()

In [119]:
clipboard_gb.shape

(37, 2)

In [120]:
clipboard_gb

,cr_by,product_id
0,261014mu@gmail.com,"{52548, 52549, 52550, 52551, 52552, 52553, 525..."
1,ajayabeyta87@gmail.com,"{3721, 4109, 4110, 4496, 274, 4115, 3731, 1682..."
2,alycat1196@gmail.com,"{37214, 37215}"
3,angelthehivesm@gmail.com,"{38274, 38275, 38536, 38537, 38538, 38539, 385..."
4,baptisteg49@gmail.com,"{24576, 24802, 24803, 24804, 24805, 24806, 248..."
5,bkim7260@gmail.com,"{44806, 44807, 44808, 44809, 44810, 44811, 448..."
6,cassandra.winegard@matthewkenneycuisine.com,"{51712, 51713, 51714, 51715, 51716, 51738, 517..."
7,chef@chefdas.com,"{7168, 7169, 7170, 3, 7, 3080, 9, 13, 3086, 30..."
8,chloe@fnahospitality.com,"{5120, 5121, 6149, 5125, 4103, 5127, 1033, 207..."
9,chris.nuciforo@outlook.com,"{36107, 36108, 36109, 36110, 36111, 36112, 361..."


def is_id(s):
    try:
        int(s)
        return True
    except ValueError:
        pass 
    return False

with open('product_clean.csv','w', encoding='utf-8') as out:
    out.write('id,cr_by,cr_date,descr,is_listed,name,price,search_field,vendor_id\n')
    with open('product.csv','r') as f:
        for line in f.readlines():
            linesplit = line.strip().split(',')
            id_str = linesplit[0]
            if is_id(id_str):
                out.write(line)

products = pd.read_csv('product_clean.csv')
products

In [199]:
product_order_gb.to_csv('product_order_gb.csv', index=False)
clipboard_gb.to_csv('clipboard_gb.csv', index=False)
basket_gb.to_csv('basket_gb.csv', index=False)

## lsh  
- user-based    
- item-based

In [185]:
from pyspark import SparkContext
import sys

In [186]:
user_dict = {}    #email -> id
up = {}    #user-product dict
data = []    #for pyspark to paralize

i = 0
for i in range(len(product_order_gb)):
    user_dict[i] = product_order_gb.iloc[i][0]
    p = sorted(list(product_order_gb.iloc[i][1])) 
    up[i] = p
    data.append((i,tuple(p)))

In [192]:
pu = {} #product-user dict
data2 = []

for user, products in up.items():
    for p in products:
        if p not in pu.keys():
            pu[p] = [user]
        else:
            pu[p].append(user)

In [193]:
len(pu)

221

In [190]:
len(up)

39

In [195]:
row_hash = {}
rp = len(pu) #>=number of unique products
ru = len(up) #>=number of unique users

# user-based
for x in range(rp): 
    f = []
    for i in range(20):
        f.append((3*x + 13*i) % 100)    #hash function
    row_hash[x] = f

def compute_sig(products): #products: a list of products
    sig = np.ones(20, int)*int('inf')  #initial signature
    for p in products:
        h = row_hash[p]
        for i in range(20):
            if h[i] < sig[i]: 
                sig[i] = h[i]
    return tuple(list(sig))

In [ ]:
sc = SparkContext()
UP = sc.parallelize(data, 1)
SIG = UP.mapValues(compute_sig)

r = 4 #number of rows in a band
b = 5 #number of bands

bands = SIG.map(lambda x: (x[1][0:r], x[0]))
for i in range(1,b):
    band = SIG.map(lambda x: (x[1][r*i:r*(i+1)], x[0]))
    bands = bands.union(band)
    
#candidate pairs: agree on all rows of >=1 bands
can = bands.groupByKey().map(lambda x: (x[0], sorted(list(x[1])))).collect()

# turn list of >= 3 users to pairs
canPairs = []
for c in can:
    if len(c[1]) == 2: canPairs.append(tuple(c[1]))
    if len(c[1]) > 2:
        for i in range(len(c[1])-1):
            for j in range(i+1, len(c[1])):
                canPairs.append((c[1][i],c[1][j]))
canPairs = list(set(canPairs)) #remove duplicate pairs

def jaccard_similarity(a,b):
    c = set(a).intersection(set(b))
    return float(len(c)) / (len(a) + len(b) - len(c))

#scan user-product dict to compute jaccard similarities of candidate pairs in canPairs
similarity = np.empty([len(canPairs),3]) #[user, user, sim]
for i in range(len(canPairs)):
    cp = canPairs[i]
    a = up[cp[0]]
    b = up[cp[1]]
    sim = jaccard_similarity(a,b)
    similarity[i] = np.array([cp[0],cp[1],sim])

In [198]:
sc.stop()

In [ ]:
recProducts = {} #key: cadidate user; value: similarity between the candidate user and the target user
for u in um.keys():
    canUsers = {}  #key: cadidate user; value: similarity between the candidate user and the target user
    for i in range(similarity.shape[0]):
        if u == similarity[i,0]:
            canUsers[similarity[i,1]] = sim
        if u == similarity[i,1]:
            canUsers[similarity[i,0]] = sim
    if len(canUsers) > 5: #sort cadidates by similarity, output top5: topUsers
        topUsers = [t[0] for t in sorted(canUsers.items(), key=lambda item: item[1], reverse=True)[:5]]
    else: topUsers = list(canUsers.keys())
    
    topProducts = {}  #key: product ordered by topUser; value: count
    for user in topUsers:
        for p in up[user]:
            topProducts[p] = topProducts.get(p,0)+1
        if len(topProducts) >5: #sort by count, output top5 product: recProd
            recProducts[p] = [m[0] for m in sorted(topProducts.items(), key=lambda item: item[1], reverse=True)[:5]]
        else:
            recProducts[p] = list(recProducts.keys())
recommend = [(p,recProducts[p]) for p in sorted(recProducts.keys())]
with open('user_based_results.txt','w') as out:
    out.writelines('%s,%s\n' % (user_dict[up[0]],str(up[1]).strip('[|]').replace(' ', '')) for up in recommend if len(up[1])>0)

user_mat_order = 
item_mat_order = 

user_mat_clipboard = 
item_mat_clipboard = 

    

def compute_jaccard_similarity(a, b):
    c = set(a).intersection(set(b))
    return float(len(c)) / (len(a) + len(b) - len(c))

def compute_user_similarity(df, t):
    user_similarity = {}
    i = 0
    for i in range(len(df)-1):
        u1 = df.iloc[i][0]
        p1 = df.iloc[i][0]
        for j in range(i+1, len(df)):
            u2 = df.iloc[j][0]
            p2 = df.iloc[j][0]
            s = compute_jaccard_similarity(p1, p2)
            if s >= t:
                if u1 not in user_similarity.keys():
                    user_similarity[u1] = [{u2: s}]
                else:
                    user_similarity[u1].append({u2: s})
                if u2 not in user_similarity.keys():
                    user_similarity[u2] = [{u1: s}]
                else:
                    user_similarity[u2].append({u1: s})
    return user_similarity

compute_user_similarity(product_order_gb)

## apriori    
- rule-based, find frequent itemsets        
- apriori: basic, but too complex if dataset is large. Confidence and lift to be updated.    
- toivonen: works for larger dataset. To be updated

In [124]:
order1 = pd.read_csv('tfe_order_backup_1128.csv')
order2 = pd.read_csv('tfe_order_1128.csv')
order2 = order2[order2['id']>1603]
order_cols = ['id','cr_by','cr_date','buyer_email','vendor_id','old_vendor_id','order_no']
order = pd.concat([order1[order_cols],order2[order_cols]], axis=0)

In [125]:
order = remove_accounts(order,'cr_by')
order = remove_accounts(order,'buyer_email')

In [134]:
order.head()

,order_id,cr_by,cr_date,buyer_email,vendor_id,old_vendor_id,order_no
3,4,chloe@fnahospitality.com,2016-09-28 22:02:01,chloe@fnahospitality.com,97.0,5.354902e+15,2016092800002
10,11,fed@thecheffed.com,2016-07-30 02:08:44,fed@thecheffed.com,107.0,6.132571e+15,2016073000003
17,18,venturelaw@gmail.com,2017-02-17 00:23:42,venturelaw@gmail.com,53.0,4.671189e+15,2017021700001
19,20,jasmine@thespringsla.com,2016-08-09 22:11:20,jasmine@thespringsla.com,98.0,5.398724e+15,2016080900002
36,37,jessecason@gmail.com,2016-08-23 19:01:13,jessecason@gmail.com,61.0,5.071555e+15,2016082300001


In [126]:
order.shape

(87, 7)

In [127]:
# top buyers by number of orders
order['buyer_email'].value_counts()

angelthehivesm@gmail.com                       24
nidhishetty19952017@gmail.com                  13
yeekai@cogcoffee.com                           11
jessecason@gmail.com                            6
venturelaw@gmail.com                            6
fed@thecheffed.com                              4
marco@marcoswesthollywood.com                   4
sathishkavinkumar@gmail.com                     4
cassandra.winegard@matthewkenneycuisine.com     2
jasmine@thespringsla.com                        2
chloe@fnahospitality.com                        2
thenickelmine@gmail.com                         1
metromarinedrivehotel@gmail.com                 1
maxnycrestaurant@gmail.com                      1
karankapur1900@gmail.com                        1
marcie@marcieps.com                             1
ivan@intelligenia.com                           1
santamonica@bodegawinebar.com                   1
sarah@sonomaorganics.com                        1
chris.nuciforo@outlook.com                      1


In [128]:
order_entry1 = pd.read_csv('order_entry_backup_1128.csv')
order_entry2 = pd.read_csv('order_entry_1128.csv')
order_entry2 = order_entry2[order_entry2['id']>525]
oe_cols = ['id','product_order_id','order_id']
order_entry = pd.concat([order_entry1[oe_cols],order_entry2[oe_cols]], axis=0)

In [137]:
order_entry.head()

,order_entry_id,product_order_id,order_id
0,1,1,59
1,2,2,55
2,3,3,12
3,4,4,12
4,5,5,61


In [133]:
order.rename(columns={'id': 'order_id'}, inplace=True)

In [135]:
order_entry.rename(columns={'id': 'order_entry_id'}, inplace=True)

In [144]:
product_order_notnull.rename(columns={'id': 'product_order_id', 'source_product_id':'product_id'}, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [142]:
basket = order[['order_id','cr_by']].merge(order_entry[['order_id','product_order_id']], on='order_id',how='left').reset_index()

In [150]:
product_order_notnull['product_order_id']=product_order_notnull['product_order_id'].map(lambda x: int(x))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [151]:
basket = basket.merge(product_order_notnull[['product_order_id','product_id']], on='product_order_id', how='left')

In [155]:
basket.shape

(374, 5)

In [153]:
basket.head()

,index,order_id,cr_by,product_order_id,product_id
0,0,4,chloe@fnahospitality.com,12,NaN
1,1,4,chloe@fnahospitality.com,18,NaN
2,2,4,chloe@fnahospitality.com,22,NaN
3,3,4,chloe@fnahospitality.com,30,NaN
4,4,4,chloe@fnahospitality.com,39,NaN


In [157]:
basket_notnull = basket[basket['product_id'].notnull()]

In [159]:
basket_gb = basket_notnull.groupby('order_id')['product_id'].apply(set).to_frame().reset_index()

In [161]:
basket_gb.shape  #70 baskets for apriori

(70, 2)

In [160]:
basket_gb.head()

,order_id,product_id
0,1002,{23306.0}
1,1003,"{22304.0, 22627.0, 22757.0, 23307.0, 22956.0, ..."
2,1552,{24752.0}
3,2752,{34061.0}
4,3202,"{35211.0, 35212.0, 35213.0, 35214.0, 35215.0, ..."


In [163]:
baskets = []
for row in basket_gb.itertuples():
    basket = list(row.product_id)
    baskets.append(basket)

n = len(baskets)

In [165]:
# find frequent 1-itemsets
def get_L1(sample, toivonen=False):
    count = {}
    for basket in sample:
        for item in basket:
            count[item] = count.get(item,0) + 1
    L1 = []
    for item, sup in count.items():
        if sup >= s:
            L1.append((item,))
    return L1


# find freq k-itemsets
def scan(dataset, Ck, minSup):
    count = {}
    for basket in dataset:
        for can in Ck:
            if set(can).issubset(basket):
                count[can] = count.get(can,0) + 1
    Lk = []
    for can, sup in count.items():
        if sup >= minSup:
            Lk.append(can)
    return Lk


# generate candidate (k+1)-itemsets from Lk
def aprioriGen(Lk, k):
    C = []
    for i in range(len(Lk)):
        for j in range(i+1, len(Lk)):
            # if L1 and L2 have (k-1) common items, combine them
            L1 = Lk[i][:k-2]
            L2 = Lk[j][:k-2]
            if L1 == L2:
                C.append(tuple(sorted(list(set(Lk[i])|set(Lk[j])))))
    return C

def apriori(sample):
    # L - all freguent itemsets
    # k=1, find frequent 1-itemsets L1
    L1 = get_L1(sample)
    L = [L1]
    L_flat = L1
    
    # k>=2, find frequent k-itemsets Lk
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        if len(Ck) == 0:
            break
        Lk = scan(sample, Ck, s)
        if len(Lk) == 0:
            break
        if len(Lk) > 0:
            L.append(Lk)
            L_flat += Lk
            k += 1
    return L_flat

In [168]:
t = 0.05  #threshold
s = n*t
fi = apriori(baskets)

In [169]:
print(fi)

[(38315.0,), (38317.0,), (38318.0,), (38320.0,), (38255.0,), (38254.0,), (38310.0,), (38311.0,), (38313.0,), (38316.0,), (38312.0,), (38314.0,), (44589.0,), (44588.0,), (40284.0,), (45716.0,), (51584.0,), (51585.0,), (51587.0,), (51576.0,), (51578.0,), (51580.0,), (51582.0,), (38315.0, 38320.0), (38255.0, 38315.0), (38254.0, 38315.0), (38255.0, 38317.0), (38254.0, 38317.0), (38255.0, 38318.0), (38254.0, 38318.0), (38255.0, 38320.0), (38254.0, 38320.0), (38254.0, 38255.0), (38313.0, 38315.0), (38315.0, 38316.0), (38255.0, 38310.0), (38255.0, 38311.0), (38255.0, 38313.0), (38255.0, 38316.0), (38254.0, 38310.0), (38254.0, 38313.0), (38254.0, 38316.0), (38313.0, 38316.0), (38255.0, 38312.0), (38255.0, 38314.0), (38254.0, 38312.0), (38254.0, 38314.0), (38313.0, 38314.0), (38314.0, 38316.0), (40284.0, 44588.0), (51584.0, 51585.0), (51584.0, 51587.0), (51576.0, 51584.0), (51578.0, 51584.0), (51580.0, 51584.0), (51582.0, 51584.0), (51585.0, 51587.0), (51576.0, 51585.0), (51578.0, 51585.0), (51